In [1]:
import cv2
import numpy as np
import os
import h5py
from tqdm.notebook import tqdm
from typing import Tuple
from PIL import Image
import matplotlib.pyplot as plt
device = 'cuda:0'

In [3]:
try:
    from core.raft_stereo import RAFTStereo
except ImportError:
    import os
    os.chdir("/RAFT-Stereo")
    from core.raft_stereo import RAFTStereo
    
FRPASS = "frames_cleanpass"
from train_fusion.dataloader import StereoDataset, StereoDatasetArgs

import torch
from torch import nn
import numpy as np
from torch.utils.data import DataLoader
from fusion_args import FusionArgs
args = FusionArgs()
args.hidden_dims = [128, 128, 128]
args.corr_levels = 4
args.corr_radius = 4
args.n_downsample = 3
args.context_norm = "batch"
args.n_gru_layers = 2
args.shared_backbone = True
args.mixed_precision = True
args.corr_implementation = "reg"
args.slow_fast_gru = False
args.restore_ckpt = "models/raftstereo-realtime.pth"


args.lr = 0.001
args.train_iters = 7
args.valid_iters = 20
args.wdecay = 0.0001
args.num_steps = 100000
args.valid_steps = 1000
args.name = "StereoFusion"
args.batch_size = 4
args.fusion = "AFF"
args.shared_fusion = True
args.freeze_backbone = []
args.both_side_train= False

In [4]:
model = torch.nn.DataParallel(RAFTStereo(args)).to(device)
model.load_state_dict(torch.load(args.restore_ckpt))
model.eval()
model = model.module

In [19]:
from typing import List


DISPARITY_MAX = 64


def cv2img_to_torch(img: np.ndarray):
    img = img.transpose(2, 0, 1)
    img = torch.tensor(img).to(device).float().unsqueeze(0)
    return img


def process_frame_spectral(frame_folder: str):
    left = cv2.imread(os.path.join(frame_folder, "left.png"))
    right = cv2.imread(os.path.join(frame_folder, "right.png"))
    left = cv2img_to_torch(left)
    right = cv2img_to_torch(right)
    _, flow = model(left, right, iters=15, test_mode=True)
    flow = -flow[0].detach().cpu().numpy().transpose(1, 2, 0)
    return flow


def store_disparity(
    frame_folder: str, channel: str, disp: np.ndarray, DISPARITY_MAX=64
):
    disparity_color = cv2.applyColorMap(
        (np.clip(disp, 0, DISPARITY_MAX) / DISPARITY_MAX * 255.0).astype(np.uint8),
        cv2.COLORMAP_MAGMA,
    )
    cv2.imwrite(os.path.join(frame_folder, channel, "disparity.png"), disparity_color)


def process_frame(frame_folder: str, overwrite=False):
    if (
        not overwrite
        and os.path.exists(os.path.join(frame_folder, "rgb", "disparity.png"))
        and os.path.exists(os.path.join(frame_folder, "nir", "disparity.png"))
    ):
        return
    disparity_rgb = process_frame_spectral(frame_folder + "/rgb")
    disparity_nir = process_frame_spectral(frame_folder + "/nir")

    disparity_max = max(disparity_rgb.max(), disparity_nir.max()) // 64 * 64
    store_disparity(frame_folder, "rgb", disparity_rgb, disparity_max)
    store_disparity(frame_folder, "nir", disparity_nir, disparity_max)
    return disparity_rgb, disparity_nir


def process_frame_spectral_batch(frame_folder_list: List[str], spectral: str):
    left_list = []
    right_list = []
    for frame_folder in frame_folder_list:
        left = cv2.imread(os.path.join(frame_folder, spectral, "left.png"))
        right = cv2.imread(os.path.join(frame_folder, spectral, "right.png"))
        left = cv2img_to_torch(left)
        right = cv2img_to_torch(right)
        left_list.append(left)
        right_list.append(right)
    left = torch.cat(left_list, 0)
    right = torch.cat(right_list, 0)
    with torch.no_grad():
        _, flow = model(left, right, iters=24, test_mode=True)
    flow = -flow[:, 0].cpu().numpy()
    return flow


def process_frame_batch(frame_folder_list: List[str], overwrite=False):

    disparity_rgb = process_frame_spectral_batch(frame_folder_list, "rgb")
    disparity_nir = process_frame_spectral_batch(frame_folder_list, "nir")

    disparity_max = (max(disparity_rgb.max(), disparity_nir.max()) + 31) // 32 * 32
    for idx, frame_folder in enumerate(frame_folder_list):
        store_disparity(frame_folder, "rgb", disparity_rgb[idx], disparity_max)
        store_disparity(frame_folder, "nir", disparity_nir[idx], disparity_max)
    return disparity_rgb, disparity_nir


def process_scene(folder: str):
    frame_folders = [
        os.path.join(folder, x)
        for x in tqdm(os.listdir(folder))
        if x.split("_")[-1].isdigit()
    ]
    print(len(frame_folders))
    frame_folders.sort()
    for frame in tqdm(frame_folders):
        try:
            process_frame(frame)
        except Exception as e:
            print(frame)
            print(e)
            continue

In [ ]:
PATH = "/bean/depth/09-04-18-40-09/18_40_08_851/post.npz"
post = np.load(PATH)
print(post["transform"])

transform_mtx = post["transform"]


In [21]:
def transform_lidarpoints_to_image(
        lidar_points: np.ndarray, width: int, height,
        focal_length: float, cx: float, cy: float,
    ):
    '''
    라이다 points 를 카메라 coordinate로 옮깁니다.
    카메라 width, height 내부의 point만 반환합니다.
    '''
    transform_matrix = transform_mtx
    lidar_points = lidar_points.reshape(-1, 3) * 1000

    # 3D 라이다 포인트를 4xN 행렬로 변환

    lidar_points = np.concatenate(
        [lidar_points, np.ones((lidar_points.shape[0], 1))], axis=1
    ).T

    # 변환 행렬을 사용하여 라이다 포인트를 카메라 좌표계로 변환
    # camera_points = transform_matrix @ lidar_points
    camera_points = np.linalg.pinv(transform_matrix) @ lidar_points

    # 카메라 좌표계의 3D 포인트를 2D 이미지 좌표로 변환
    u = camera_points[0] * focal_length / camera_points[2] + cx
    v = camera_points[1] * focal_length / camera_points[2] + cy
    depth = camera_points[2]

    camera_surface = np.stack([u, v, depth], axis=1)
    lidar_points = lidar_points.T

    csf = camera_surface[
        (camera_surface[:, 0] > 0)
        & (camera_surface[:, 0] < width)
        & (camera_surface[:, 1] > 0)
        & (camera_surface[:, 1] < height)
        & (camera_surface[:, 2] > 0)
    ]


    csf = csf[np.argsort(csf[:, 2])[::-1]]
    return csf
    
def render_2dpoint_to_image(
        points: np.ndarray,  width: int, height: int, use_color: bool = True, MAX_DEPTH: float = 10000
    ):
    '''
    projected 2d lidar points 를 하나의 이미지로 변환합니다.
    '''
    
    if use_color:
        colormap = cv2.applyColorMap(
            np.linspace(0, 255, 256).astype(np.uint8), cv2.COLORMAP_MAGMA
        )

    canvas = (
        np.zeros((height, width, 3), dtype=np.uint8)
        if use_color
        else np.zeros((height, width), dtype=np.float32)
    )
    
    if not use_color:
        u = points[:, 0].astype(int)
        v = points[:, 1].astype(int)
        depth = points[:, 2]
        u = np.clip(u, 0, width - 3)
        v = np.clip(v, 0, height - 3)
        
        for i in range(3):
            for j in range(3):
                canvas[v + i, u + j] = depth
        
        return canvas

    for u, v, depth in points:
        radius = 3
        u = int(int(u) // 4 * 4)
        v = int(int(v) // 4 * 4)
        if use_color:
            depth_color = int(np.clip(depth / MAX_DEPTH * 255, 0, 255))

            # radius = int(depth / MAX_DEPTH * 10 + 5)
            r, g, b = map(int, colormap[depth_color][0])
            cv2.circle(canvas, (u, v), radius, (r, g, b), -1)
        else:
            for i in range(-radius, radius + 1):
                for j in range(-radius, radius + 1):
                    if (
                        0 <= int(v) + i < height
                        and 0 <= int(u) + j < width
                        and np.linalg.norm([i, j]) <= radius
                    ):
                        canvas[int(v) + i, int(u) + j] = depth


    colorbar = cv2.resize(colormap, (50, height))
    return np.concatenate([canvas, colorbar], axis=1)

        
def disparity_to_depth(disparity: np.ndarray, focal_length: float, baseline: float):
    '''
    disparity map을 depth map으로 변환합니다
    '''
    disparity = disparity.astype(np.float32)
    depth = focal_length * baseline / disparity
    depth[depth < 0] = 0
    depth[np.isnan(depth)] = 0
    depth[np.isinf(depth)] = 0
    return depth

In [22]:
def process_lidar_frame(lidar_points: np.ndarray, calibration: dict):
    '''
    라이다 포인트를 칼리브레이션을 바탕으로 처리합니다.
    '''
    focal_length: float = calibration["mtx_left"][0,0]
    cx: float = calibration["mtx_left"][0,2]
    cy: float = calibration["mtx_left"][1,2]

    image_size: Tuple[int, int] = calibration["image_size"]
    
    lidar_camera_points = transform_lidarpoints_to_image(lidar_points, image_size[0], image_size[1], focal_length, cx, cy)
    
    return lidar_camera_points

def process_lidar_h5file(h5file: str, overwrite: bool = False):
    '''
    h5file 내부의 모든 lidar points에 대해 카메라 좌표계로 변환하고, 이미지에 렌더링합니다.
    '''
    with h5py.File(h5file, "a") as f:
        calibration = f["calibration"].attrs
        depth_median = 0
        keys = list(f['frame'].keys())
        if not overwrite and "projected_points" in f.require_group(f"frame/{keys[-1]}")["lidar"]:
            f.close()
            return
        for frame in tqdm(f["frame"]):
            
            if not "image_size" in calibration:
                image = Image.open(os.path.join(os.path.dirname(h5file), frame, "rgb","left.png"))
                calibration["image_size"] = image.size
            frame = f.require_group(f"frame/{frame}")
            if "projected_points" in frame["lidar"]:
                continue
            lidar_points = frame["lidar/points"][:]
            
            lidar_projected_points = process_lidar_frame(lidar_points, calibration)
            depth_median = np.max([depth_median, np.median(lidar_projected_points[:,2])])
            if "projected_points" in frame["lidar"]:
                del frame["lidar/projected_points"]
            frame.create_dataset("lidar/projected_points", data=lidar_projected_points)
        
        for frame in f["frame"]:
            if os.path.exists(os.path.join(os.path.dirname(h5file), frame, "lidar.png")):
                continue
            frame_group = f.require_group(f"frame/{frame}")
            lidar_projected_points = frame_group["lidar/projected_points"][:]
            width, height = calibration["image_size"]
            rendered_image = render_2dpoint_to_image(lidar_projected_points, width,height, use_color=True, MAX_DEPTH=depth_median*5)
            scene_folder = os.path.dirname(h5file)
            cv2.imwrite(os.path.join(scene_folder, frame, "lidar.png"), rendered_image)
            
            # disparity_rgb = frame_group["disparity"]["rgb"][:]
            # focal_length = calibration["mtx_left"][0,0]
            # depth = disparity_to_depth(disparity_rgb, focal_length, np.linalg.norm(calibration["T"][:]))
            # depth = (np.clip(depth, 0, depth_median*5) / (depth_median*5) * 255.0).astype(np.uint8)
            # depth = cv2.applyColorMap(depth, cv2.COLORMAP_MAGMA)
            # cv2.imwrite(os.path.join(scene_folder, frame, "depth.png"), depth)
        f.close()
            


In [23]:
def frame_create_fig_png(
    scene_folder: str,
    frame_id: str,
    frame: h5py.Group,
    focal_length: float,
    baseline: float,
    use_numpy=False,
):
    """
    Frame에 대해 3x3 이미지 그리드를 생성합니다.
    각 채널의 Stereo Image, Disparity, Depth 그리고 Lidar Projected Depth를 표시합니다.
    """
    rgb_left = cv2.imread(os.path.join(scene_folder, frame_id, "rgb", "left.png"))
    rgb_right = cv2.imread(os.path.join(scene_folder, frame_id, "rgb", "right.png"))
    nir_left = cv2.imread(os.path.join(scene_folder, frame_id, "nir", "left.png"))
    nir_right = cv2.imread(os.path.join(scene_folder, frame_id, "nir", "right.png"))
    rgb_disparity = frame["disparity/rgb"][:]
    nir_disparity = frame["disparity/nir"][:]

    ########### Mis aligned frame remove
    if rgb_disparity.mean() < 64 and nir_disparity.mean() > 64:
        return

    rgb_depth = disparity_to_depth(rgb_disparity, focal_length, baseline)
    nir_depth = disparity_to_depth(nir_disparity, focal_length, baseline)
    lidar_depth = frame["lidar/projected_points"][:]

    if use_numpy:
        fig = frame_create_fig_png_numpy(
            (rgb_left, rgb_right, rgb_disparity),
            (nir_left, nir_right, nir_disparity),
            lidar_depth,
            focal_length, baseline
        )
        cv2.imwrite(os.path.join(scene_folder, frame_id, "fig.png"), fig)
        return
        # Prepare layout for 3x3 image grid
    fig, axs = plt.subplots(3, 3, figsize=(25, 15))

    depth_max = min(
        max(20000, max(np.median(rgb_depth), np.median(nir_depth)) * 3), 50000
    )
    disparity_max = min(rgb_disparity.max(), nir_disparity.max()) * 0.8

    # First row: RGB images
    axs[0, 0].imshow(cv2.cvtColor(rgb_left, cv2.COLOR_BGR2RGB))
    axs[0, 0].set_title("RGB Left")
    axs[0, 0].axis("off")

    axs[0, 1].imshow(cv2.cvtColor(rgb_right, cv2.COLOR_BGR2RGB))
    axs[0, 1].set_title("RGB Right")
    axs[0, 1].axis("off")

    # RGB Disparity with magma colormap
    im_rgb_disp = axs[0, 2].imshow(
        np.clip(rgb_disparity, 0, disparity_max),
        cmap="magma",
        vmin=0,
        vmax=disparity_max,
    )
    axs[0, 2].set_title("RGB Disparity")
    fig.colorbar(im_rgb_disp, ax=axs[0, 2])
    axs[0, 2].axis("off")

    # Second row: NIR images
    axs[1, 0].imshow(cv2.cvtColor(nir_left, cv2.COLOR_BGR2RGB))
    axs[1, 0].set_title("NIR Left")
    axs[1, 0].axis("off")

    axs[1, 1].imshow(cv2.cvtColor(nir_right, cv2.COLOR_BGR2RGB))
    axs[1, 1].set_title("NIR Right")
    axs[1, 1].axis("off")

    # NIR Disparity with magma colormap
    im_nir_disp = axs[1, 2].imshow(
        np.clip(nir_disparity, 0, disparity_max),
        cmap="magma",
        vmin=0,
        vmax=disparity_max,
    )
    axs[1, 2].set_title("NIR Disparity")
    fig.colorbar(im_nir_disp, ax=axs[1, 2])
    axs[1, 2].axis("off")
    rgb_depth[rgb_disparity < 1] = 0
    nir_depth[nir_disparity < 1] = 0
    # Third row: Depth images
    im_rgb_depth = axs[2, 0].imshow(
        np.clip(rgb_depth, 0, depth_max), cmap="magma", vmin=0, vmax=depth_max
    )
    axs[2, 0].set_title("RGB Depth")
    fig.colorbar(im_rgb_depth, ax=axs[2, 0])
    axs[2, 0].axis("off")
    im_nir_depth = axs[2, 1].imshow(
        np.clip(nir_depth, 0, depth_max), cmap="magma", vmin=0, vmax=depth_max
    )
    axs[2, 1].set_title("NIR Depth")
    fig.colorbar(im_nir_depth, ax=axs[2, 1])
    axs[2, 1].axis("off")
    # Lidar depth (point cloud projected)
    u, v = lidar_depth[:, 0], -lidar_depth[:, 1]
    z = lidar_depth[:, 2]
    sc = axs[2, 2].scatter(
        u, v, c=np.clip(z, 0, depth_max), cmap="magma", vmin=0, vmax=depth_max
    )
    axs[2, 2].set_title("Lidar Depth")
    fig.colorbar(sc, ax=axs[2, 2])
    axs[2, 2].axis("off")
    # Display the full layout
    # plt.tight_layout()
    # plt.show()
    plt.savefig(os.path.join(scene_folder, frame_id, "fig.png"))
    plt.close()


def process_h5file_create_figs(h5path: str):
    """
    h5file의 모든 frame에 대해 fig png 이미지들을 생성합니다.
    """
    scene_id = os.path.dirname(h5path).split("/")[-1]
    with h5py.File(h5path, "r") as f:
        focal_length = f["calibration"].attrs["mtx_left"][0, 0]
        baseline = np.linalg.norm(f["calibration"].attrs["T"][:])
        for frame_id in f["frame"]:
            frame_create_fig_png(
                os.path.dirname(h5path),
                frame_id,
                f["frame"].require_group(frame_id),
                focal_length,
                baseline,
            )

In [24]:
import time


rgb_label = None
nir_label = None
colorbar_disparity = None

def frame_create_fig_png_numpy(
    rgb_tuple: Tuple[np.ndarray, np.ndarray, np.ndarray],
    nir_tuple: Tuple[np.ndarray, np.ndarray, np.ndarray],
    lidar_depth: np.ndarray,
    focal_length : float, baseline: float,
):
    rgb_left, rgb_right, rgb_disparity = rgb_tuple
    H,W = rgb_left.shape[:2]
    nir_left, nir_right, nir_disparity = nir_tuple
    rgb_disparity = rgb_disparity[:H, :W]
    nir_disparity = nir_disparity[:H, :W]
    #depth_max = min(max(20000, max(np.median(rgb_depth), np.median(nir_depth)) * 3), 50000)
    disparity_max = min(rgb_disparity.max(), nir_disparity.max()) * 0.8
    rgb_disparity_color = cv2.applyColorMap(
        (np.clip(rgb_disparity, 0, disparity_max) / disparity_max * 255.0).astype(
            np.uint8
        ),
        cv2.COLORMAP_MAGMA,
    )
    nir_disparity_color = cv2.applyColorMap(
        (np.clip(nir_disparity, 0, disparity_max) / disparity_max * 255.0).astype(
            np.uint8
        ),
        cv2.COLORMAP_MAGMA,
    )
    rgb_concat = np.concatenate([rgb_left, rgb_right, rgb_disparity_color], axis=1)
    nir_concat = np.concatenate([nir_left, nir_right, nir_disparity_color], axis=1)
    
    colorbar_disparity = np.linspace(0, 255,256).reshape(256,1).astype(np.uint8)
    colorbar_disparity = cv2.applyColorMap(colorbar_disparity, cv2.COLORMAP_MAGMA)
    colorbar_disparity = cv2.resize(colorbar_disparity, (50, rgb_left.shape[0]))
    colorbar_disparity_text = np.zeros((rgb_left.shape[0], 100, 3), dtype=np.uint8) + 255
    for i in range(0, rgb_left.shape[0], 100):
        cv2.putText(
            colorbar_disparity_text,
            str(int(i / rgb_left.shape[0] * disparity_max)),
            (10, i+15),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (0, 0, 0),
            1,
        )
    colorbar_depth_text = np.zeros((rgb_left.shape[0], 100, 3), dtype=np.uint8) + 255
    for i in range(100, rgb_left.shape[0], 100):
        cv2.putText(
            colorbar_depth_text,
            str(round(focal_length * baseline / (   i / rgb_left.shape[0] * disparity_max)/1000,1)) + "m",
            (10, i+15),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (0, 0, 0),
            1,
        )
    rgb_concat = np.concatenate([rgb_concat, colorbar_disparity, colorbar_disparity_text], axis=1)
    nir_concat = np.concatenate([nir_concat, colorbar_disparity, colorbar_disparity_text], axis=1)
    lidar_depth[:,2] = focal_length * baseline / lidar_depth[:,2]
    lidar_disparity = render_2dpoint_to_image(lidar_depth, rgb_left.shape[1], rgb_left.shape[0], use_color=False)
    lidar_disparity_color = cv2.applyColorMap(
        (np.clip(lidar_disparity, 0, disparity_max) / disparity_max * 255.0).astype(np.uint8),
        cv2.COLORMAP_MAGMA,
    )
    rgb_concat = np.concatenate([rgb_concat, lidar_disparity_color, colorbar_disparity, colorbar_depth_text], axis=1)
    nir_padding = np.zeros((nir_concat.shape[0], rgb_concat.shape[1] - nir_concat.shape[1], 3), dtype=np.uint8) + 255
    nir_concat = np.concatenate([nir_concat, nir_padding], axis=1)
    global rgb_label, nir_label
    if rgb_label is None:
        rgb_label = np.zeros((100, rgb_concat.shape[1], 3), dtype=np.uint8) + 255
        cv2.putText(rgb_label, "RGB Left", (W // 2, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 1)
        cv2.putText(rgb_label, "RGB Right", (W // 2 * 3, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 1)
        cv2.putText(rgb_label, "RGB Disparity", (W // 2 * 5, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 1)
        cv2.putText(rgb_label, "Lidar Depth", (W // 2 * 7, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 1)
        nir_label = np.zeros((100, rgb_concat.shape[1], 3), dtype=np.uint8) + 255
        cv2.putText(nir_label, "NIR Left", (W // 2, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 1)
        cv2.putText(nir_label, "NIR Right", (W // 2 * 3, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 1)
        cv2.putText(nir_label, "NIR Disparity", (W // 2 * 5, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 1)
        

    fig = np.concatenate([rgb_label, rgb_concat, nir_label, nir_concat], axis=0)
    return fig
    
    
    
    
    

In [25]:
import threading
import time
from typing import Union
from IPython.display import Video, display

def save_video(image_paths: list[str], video_path, fps=5):
    '''
    image_paths로 읽어 온 이미지 파일들을 순서대로 비디오 파일로 저장합니다.
    image_paths: 이미지 파일 경로 리스트
    video_path: 저장할 비디오 파일 경로
    fps: 초당 프레임 수
    '''
    print(image_paths[0])
    height, width = cv2.imread(image_paths[0]).shape[:2]
    # 비디오 파일 쓰기 설정
    fourcc = cv2.VideoWriter.fourcc(*'mp4v')  # 코덱 설정
    os.makedirs(os.path.dirname(video_path), exist_ok=True)
    out = cv2.VideoWriter(video_path, fourcc, fps, (width, height))
    
    for img in tqdm(image_paths):
        img = img.replace("frames_cleanpass", "plot")
        if not os.path.exists(img):
            break
        image = cv2.imread(img)
        out.write( image)
    
    out.release()
    print(f"비디오가 저장되었습니다: {video_path}")


def process_h5_plt_video(h5file: str, create_fig=False):
    '''
    h5file의 모든 frame에 대해 fig.png 이미지들을 하나의 비디오로 저장합니다.
    '''
    with h5py.File(h5file, "r")  as f:
        frame_ids = list(f["frame"].keys())
        frame_id_filtered = []
        focal_length = f["calibration"].attrs["mtx_left"][0,0]
        baseline = np.linalg.norm(f["calibration"].attrs["T"][:])
        if create_fig:
            fig_create_threads = []
            for idx, frame_id in enumerate(tqdm(frame_ids)):
                thread = threading.Thread(target=frame_create_fig_png, args=(os.path.dirname(h5file), frame_id, f["frame"].require_group(frame_id), focal_length, baseline,  True))
                thread.start()
                fig_create_threads.append(thread)
                if len(fig_create_threads) >= 6 or idx == len(frame_ids) - 1:
                    for thread in fig_create_threads:
                        thread.join()
                    fig_create_threads = []        
        for frame_id in frame_ids:
            if not os.path.exists(os.path.join(os.path.dirname(h5file), frame_id, "fig.png")):
                continue
            if cv2.imread(os.path.join(os.path.dirname(h5file), frame_id, "fig.png")) is None:
                continue
            frame_id_filtered.append(frame_id)
        image_paths = [os.path.join(os.path.dirname(h5file), x, "fig.png") for x in frame_id_filtered]
        save_video(image_paths, h5file.replace(".hdf5", ".mp4"))
        f.close()

In [26]:
def stereo_depth_lidar_loss(frame: h5py.Group, focal_length: float, baseline: float):
    disparity_rgb = frame["disparity/rgb"][:]
    disparity_nir = frame["disparity/nir"][:]
    projected_points = frame["lidar/projected_points"][:]

    u = projected_points[:, 0]
    v = projected_points[:, 1]
    z = focal_length * baseline / projected_points[:, 2]
    depth_rgb = disparity_rgb[v.astype(int), u.astype(int)].squeeze()
    depth_nir = disparity_nir[v.astype(int), u.astype(int)].squeeze()
    
    rsme_rgb = np.sqrt(np.mean((depth_rgb - z) ** 2))
    rsme_nir = np.sqrt(np.mean((depth_nir - z) ** 2))
    mae_rgb = np.mean(np.abs(depth_rgb - z))
    mae_nir = np.mean(np.abs(depth_nir - z))
    return (rsme_rgb, rsme_nir), (mae_rgb, mae_nir)


def process_depth_loss_h5file(h5file: str, overwrite_loss=False):
    """
    h5file의 모든 frame에 대해
    raft stereo로 계산한 depth와 라이다 depth의 차이를 계산합니다.
    rsme와 mae를 반환합니다.
    """
    with h5py.File(h5file, "a") as f:
        frame_ids = list(f["frame"].keys())
        focal_length = f["calibration"].attrs["mtx_left"][0, 0]
        baseline = np.linalg.norm(f["calibration"].attrs["T"][:])
        __rsme_rgb = []
        __rsme_nir = []
        __mae_rgb = []
        __mae_nir = []
        if not overwrite_loss and "rsme_rgb" in f.attrs:
            f.close()
            return
        for frame_id in tqdm(frame_ids):
            frame = f.require_group(f"frame/{frame_id}")
            if "lidar/projected_points" not in frame:
                continue
            (rsme_rgb, rsme_nir), (mae_rgb, mae_nir) = stereo_depth_lidar_loss(
                frame, focal_length, baseline
            )
            frame["disparity/rgb"].attrs["rsme"] = rsme_rgb
            frame["disparity/nir"].attrs["rsme"] = rsme_nir
            frame["disparity/rgb"].attrs["mae"] = mae_rgb
            frame["disparity/nir"].attrs["mae"] = mae_nir
            __rsme_rgb.append(rsme_rgb)
            __rsme_nir.append(rsme_nir)
            __mae_rgb.append(mae_rgb)
            __mae_nir.append(mae_nir)

        f.attrs["rsme_rgb"] = np.mean(__rsme_rgb)
        f.attrs["rsme_nir"] = np.mean(__rsme_nir)
        f.attrs["mae_rgb"] = np.mean(__mae_rgb)
        f.attrs["mae_nir"] = np.mean(__mae_nir)
        
        plt.plot(__rsme_rgb, label="RSME RGB")
        plt.plot(__rsme_nir, label="RSME NIR")
        plt.legend()
        plt.show()
        
        f.close()

In [27]:
def flush_queue(queue: list[h5py.Group], frame_ids: list[str]):
    disparity_rgb, disparity_nir = process_frame_batch(frame_ids)
    for idx, frame in enumerate(queue):
        if "disparity/rgb" in frame:
            del frame["disparity/rgb"]
        if "disparity/nir" in frame:
            del frame["disparity/nir"]
        frame.create_dataset("disparity/rgb", data=disparity_rgb[idx])
        frame.create_dataset("disparity/nir", data=disparity_nir[idx])
        
        if disparity_nir[idx].mean() > 64 and disparity_rgb[idx].mean() < 64:
            frame.attrs["align_error"] = True
            print(f"A frame {frame} has an alignment error")
        else:
            frame.attrs["align_error"] = False
    queue.clear()
    frame_ids.clear()


def process_disparity_h5file(h5file: str, overwrite=False, batch_size = 5):
    '''
    h5file의 모든 frame에 대해
    Raft Stereo 모델을 사용하여 disparity를 추출하고 h5 파일에 저장합니다.
    disparity를 color map으로 변환하여 png 파일로 저장합니다.
    '''
    
    process_queue = []
    process_queue_ids = []
    
    
    with h5py.File(h5file, "a")  as f:
        frame_ids = list(f["frame"].keys())
        if not overwrite and "disparity" in f["frame"][frame_ids[-1]]:
            f.close()
            return
        for frame_id in tqdm(frame_ids):
            frame = f.require_group(f"frame/{frame_id}")
            if not overwrite and "disparity" in frame:
                continue
            process_queue.append(frame)
            process_queue_ids.append(os.path.join(os.path.dirname(h5file), frame_id))
            
            if len(process_queue) >= batch_size or frame_id == frame_ids[-1]:
                flush_queue(process_queue, process_queue_ids)
            continue
            output = process_frame(os.path.join(os.path.dirname(h5file), frame_id))
            if output is None:
                continue
            disparity_rgb, disparity_nir = output
            
            frame.create_dataset("disparity/rgb", data=disparity_rgb)
            frame.create_dataset("disparity/nir", data=disparity_nir)
            if not "disparity" in frame:
                continue
            disparity_rgb = frame["disparity/rgb"][:]
            disparity_nir = frame["disparity/nir"][:]
            if disparity_nir.mean() > 64 and disparity_rgb.mean() < 64:
                frame.attrs["align_error"] = True
                print(f"A frame {frame_id} has an alignment error")
            else:
                frame.attrs["align_error"] = False
        f.close()

In [ ]:
from myutils.hy5py import (
    calibration_property,
    get_frame_by_path,
    read_calibration,
    read_lidar,
)
from myutils.image_process import read_image_pair, cv2toTensor
import cv2
import matplotlib.pyplot as plt

from myutils.points import (
    combine_disparity_by_edge,
    combine_disparity_by_lidar,
    lidar_points_to_disparity,
)

frame_path = "/bean/depth/09-08-17-27-33/17_32_56_438"
rgb_left, rgb_right, nir_left, nir_right = read_image_pair(frame_path)

calibration = read_calibration(os.path.dirname(frame_path) + "/0.hdf5")
focal_length, baseline, cx, cy = calibration_property(calibration)
transform_mtx = np.load("jai_transform.npy")
with get_frame_by_path(frame_path) as frame:
    lidar_disparity = lidar_points_to_disparity(
        frame["lidar/points"][:] * 1000,
        transform_mtx,
        focal_length,
        baseline,
        cx,
        cy,
    )
    flow_rgb: np.ndarray = frame["disparity/rgb"][:] # type: ignore
    flow_nir: np.ndarray = frame["disparity/nir"][:] # type: ignore
    depth_mono_rgb: np.ndarray = frame["depth_mono/rgb"][:] # type: ignore
    depth_mono_nir: np.ndarray = frame["depth_mono/nir"][:] # type: ignore

disparity_combined = combine_disparity_by_lidar(
    lidar_disparity, flow_rgb, flow_nir, 24, 24
)
disparity_combined_2 = combine_disparity_by_edge(
    lidar_disparity, flow_rgb, flow_nir, rgb_left, nir_left, 24, 24
)

print(disparity_combined.mean(), disparity_combined.max(), disparity_combined.shape, disparity_combined.dtype)

plt.figure(figsize=(20, 8))
plt.subplot(241)
plt.imshow(rgb_left)
plt.subplot(242)
plt.imshow(flow_rgb, cmap="magma", vmin=0, vmax=32)
plt.subplot(243)
plt.imshow(depth_mono_rgb, cmap="magma", vmin=0, vmax=32)
plt.subplot(244)
plt.imshow(disparity_combined, cmap="magma", vmin=0, vmax=32)
plt.subplot(245)
plt.imshow(nir_left, cmap="gray")
plt.subplot(246)
plt.imshow(flow_nir, cmap="magma", vmin=0, vmax=32)
plt.subplot(247)
plt.imshow(depth_mono_nir, cmap="magma", vmin=0, vmax=32)
plt.subplot(248)
plt.imshow(disparity_combined_2, cmap="magma", vmin=0, vmax=32)
plt.show()

In [ ]:
def process_h5file_all(
    h5file: str,
    overwrite_disparity=False,
    overwrite_loss=False,
    overwrite_plot=False,
    overwrite_lidar=False,
):
    """
    h5file에 대해 disparity, plot, lidar를 모두 처리합니다.
    """
    process_disparity_h5file(h5file, overwrite=overwrite_disparity)
    process_lidar_h5file(h5file, overwrite=overwrite_lidar)
    process_depth_loss_h5file(h5file, overwrite_loss=overwrite_loss)
    if overwrite_plot or not os.path.exists(h5file.replace(".hdf5", ".mp4")):
        process_h5_plt_video(h5file, create_fig=True)

    print(f"Finished processing {h5file}")

In [ ]:
process_disparity_h5file("/bean/depth/09-10-14-59-19/0.hdf5", overwrite=True)

In [38]:
from tqdm.notebook import tqdm
import h5py
import cv2
from myutils.points import lidar_points_to_disparity_with_cal, refine_disparity_points
from numpy.lib.stride_tricks import sliding_window_view
import numpy as np
from myutils.image_process import read_image_pair
from myutils.hy5py import read_calibration, get_frame_by_path
import torch

tmtx = np.load("jai_transform.npy")


def evaluate_exposure(image):
    if image.mean() < 12:
        return -1
    if image.mean() > 196:
        return 1
    return 0


def evaluate_exposure_stereo(image_left, image_right):
    left_intensity = image_left.mean()
    right_intensity = image_right.mean()
    if left_intensity / right_intensity > 1.15:
        return 1
    if right_intensity / left_intensity > 1.15:
        return -1
    return 0


def fft_blur_detection(image):
    """
    Detect blur in an image using FFT (Fast Fourier Transform).
    Steps:
    1. Convert image to grayscale.
    2. Apply Fourier Transform.
    3. Analyze high frequency components to determine blur level.

    Parameters:
    - image: Input BGR image

    Returns:
    - blur_metric: A scalar representing the amount of blur in the image.
                   Lower values indicate more blur.
    """
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape) == 3 else image

    # Get the image dimensions
    (h, w) = gray.shape

    # Apply a Fast Fourier Transform (FFT)
    f_transform = np.fft.fft2(gray)
    f_shifted = np.fft.fftshift(f_transform)

    # Compute the magnitude spectrum
    magnitude_spectrum = 20 * np.log(np.abs(f_shifted))

    # Crop the magnitude spectrum to remove the low frequencies
    # Center coordinates
    center_row, center_col = h // 2, w // 2
    # Radius of the low frequency region to remove
    radius = 60

    # Create a mask to zero out the low frequencies
    mask = np.ones((h, w), np.uint8)
    cv2.circle(mask, (center_col, center_row), radius, 0, -1)

    # Apply the mask to the magnitude spectrum
    f_shifted_masked = f_shifted * mask

    # Compute the inverse FFT
    f_ishift = np.fft.ifftshift(f_shifted_masked)
    reconstructed_image = np.fft.ifft2(f_ishift)
    reconstructed_image = np.abs(reconstructed_image)

    # Compute the variance of the reconstructed image (sharpness measure)
    blur_metric = np.var(reconstructed_image)

    return blur_metric


def lidar_box_error(lidar_points, disparity_map, box_size=5):
    """
    벡터화된 방식으로 LiDAR 포인트와 Disparity Map을 이용하여 Box Error를 계산합니다.

    Parameters:
    - lidar_points: (N, 3) 배열로 각 행이 (u, v, d)를 나타냅니다.
    - disparity_map: 2D 배열로 Disparity Map을 나타냅니다.
    - box_size: 박스의 크기 (기본값은 5).

    Returns:
    - mean_box_error: 모든 포인트에 대한 평균 Box Error.
    - errors: 각 포인트에 대한 [u, v, box_error] 배열.
    """
    # LiDAR 포인트 분리
    u, v, d = lidar_points.T
    h, w = disparity_map.shape
    half_box = box_size // 2

    # Disparity Map을 패딩하여 경계 문제를 해결
    padded_disparity = np.pad(disparity_map, 
                              pad_width=((half_box, half_box), (half_box, half_box)), 
                              mode='edge')

    # 슬라이딩 윈도우을 이용해 모든 5x5 박스를 생성
    windows = sliding_window_view(padded_disparity, (box_size, box_size))
    # windows의 shape: (h, w, box_size, box_size)

    # u와 v가 이미지 경계를 벗어나지 않도록 클리핑
    u_clipped = np.clip(u, 0, w - 1).astype(int)
    v_clipped = np.clip(v, 0, h - 1).astype(int)

    # 각 포인트에 해당하는 박스 추출
    # indexing을 통해 (N, box_size, box_size) 형태의 배열을 얻음
    boxes = windows[v_clipped, u_clipped, :, :]

    # 실제 d 값과 박스 내 값들의 절대 차이 계산
    diffs = np.abs(boxes - d[:, np.newaxis, np.newaxis])

    # 박스 에러 (절대 차이의 평균) 계산
    box_errors = np.mean(diffs, axis=(1, 2))

    # NaN 값이 있는 경우 제거
    valid = ~np.isnan(box_errors)
    box_errors = box_errors[valid]
    u_valid = u_clipped[valid]
    v_valid = v_clipped[valid]

    # 전체 평균 박스 에러 계산
    mean_box_error = np.mean(box_errors)

    # 각 포인트의 [u, v, box_error] 배열 생성
    errors = np.stack([u_valid, v_valid, box_errors], axis=1)

    return mean_box_error, errors


def evaluate_frame_exposure(frame: h5py.Group, frame_path: str):
    images = read_image_pair(frame_path)
    exposure_status = [evaluate_exposure(image) for image in images]
    stereo_status = [
        evaluate_exposure_stereo(images[0], images[1]),
        evaluate_exposure_stereo(images[2], images[3]),
    ]
    #blur_status = [fft_blur_detection(image) for image in images]
    if (
        (exposure_status[0] != 0 and exposure_status[2] != 0)
        or stereo_status[0] != 0
        or stereo_status[1] != 0
    ):
        frame.attrs["exposure_error"] = True
    else:
        frame.attrs["exposure_error"] = False
    # if blur_status[2] < 120 or blur_status[3] < 120:
    #     frame.attrs["blur_error"] = True
    # else:
    #     frame.attrs["blur_error"] = False


cal = read_calibration("/bean/depth/09-09-19-46-45/0.hdf5")


def evaluate_lidar(frame: h5py.Group, frame_path: str):
    images = read_image_pair(frame_path)
    lidar_disparity = lidar_points_to_disparity_with_cal(
        frame["lidar/points"][:], tmtx, cal
    )
    lidar_disparity = refine_disparity_points(torch.from_numpy(lidar_disparity)).numpy()
    if "disparity/rgb" in frame:
        disparity_rgb = frame["disparity/rgb"][:]
        disparity_nir = frame["disparity/nir"][:]
    else:
        with torch.no_grad():
            disparity_rgb = (
                -model(
                    torch.from_numpy(images[0]).permute(2, 0, 1).unsqueeze(0).cuda(),
                    torch.from_numpy(images[1]).permute(2, 0, 1).unsqueeze(0).cuda(),
                    test_mode=True,
                )[1][0, 0]
                .cpu()
                .numpy()
            )
            disparity_nir = (
                -model(
                    torch.from_numpy(images[2]).unsqueeze(0).unsqueeze(0).repeat(1,3,1,1).cuda(),
                    torch.from_numpy(images[3]).unsqueeze(0).unsqueeze(0).repeat(1,3,1,1).cuda(),
                    test_mode=True,
                )[1][0, 0]
                .cpu()
                .numpy()
            )
    lidar_box_rgb, _ = lidar_box_error(lidar_disparity, disparity_rgb)
    lidar_box_nir, _ = lidar_box_error(lidar_disparity, disparity_nir)

    frame.attrs["lidar_align_error"] = lidar_box_rgb > 2.5 and lidar_box_nir > 2.5


def update_frame_exposure_h5(h5file: str):
    e_cnt = 0
    l_cnt = 0
    with h5py.File(h5file, "a") as f:
        last_frame = list(list(f['frame'].keys()))[-1]
        if "lidar_align_error" in f["frame"][last_frame].attrs:
            f.close()
            return
        for frame_id in tqdm(f["frame"]):
            
            frame = f["frame"][frame_id]
            
            evaluate_frame_exposure(
                frame, os.path.join(os.path.dirname(h5file), frame_id)
            )
            evaluate_lidar(frame, os.path.join(os.path.dirname(h5file), frame_id))
            if frame.attrs["exposure_error"]:
                e_cnt += 1

            if frame.attrs["lidar_align_error"]:
                l_cnt += 1
        f.close()
    print(f"Exposure Error : {e_cnt} LidarAlignError : {l_cnt}")

In [ ]:
folders = [x for x in os.listdir("/bean/depth") if x.startswith("09-20")]
for folder in folders:
    print(f"Processing {folder}")
    h5files = [os.path.join("/bean/depth", folder, x) for x in os.listdir(os.path.join("/bean/depth", folder)) if x.endswith(".hdf5")]
    h5files.sort(key=lambda x: int(os.path.basename(x).split(".")[0]))
    for h5file in h5files:
        print(f"Processing {h5file}")
        process_h5file_all(h5file)

In [39]:

FOLDERS = [x for x in os.listdir("/bean/depth") if os.path.isdir( os.path.join("/bean/depth",x))]
FOLDERS = [x for x in FOLDERS]
FOLDERS.sort()
h5files_pending = []
for folder in FOLDERS:
    print(folder)
    h5files = os.listdir(os.path.join("/bean/depth",folder))
    h5files = [x for x in h5files if x.endswith(".hdf5")]
    h5files.sort(key=lambda x : int(x.split('.')[0]) )
    for h5 in h5files:
        h5files_pending.append(os.path.join("/bean/depth",folder,h5))
print(len(h5files_pending))
for h5 in tqdm(h5files_pending):
    print(h5)
    update_frame_exposure_h5(h5)
    #process_h5file_all(h5)

09-05-17-07-36
09-05-17-10-57
09-05-17-27-03
09-05-17-27-17
09-05-17-29-14
09-05-17-32-37
09-05-17-34-15
09-05-17-34-36
09-08-17-27-33
09-09-15-21-02
09-09-19-19-31
09-09-19-26-29
09-09-19-31-19
09-09-19-46-45
09-09-20-03-55
09-09-20-04-34
09-10-14-38-20
09-10-14-43-20
09-10-14-48-46
09-10-14-54-15
09-10-14-59-19
09-10-15-09-25
09-10-15-09-53
09-10-15-11-59
09-10-15-12-08
09-10-15-12-28
09-10-15-14-14
09-10-15-16-19
09-10-15-18-33
09-10-15-21-40
09-10-15-21-52
09-20-13-45-57
09-20-13-50-44
09-20-13-53-33
09-20-13-54-41
09-28-16-44-59
09-28-16-48-17
09-28-17-06-04
09-28-17-34-59
09-28-21-15-01
09-28-21-15-50
09_28_17_58
10-01-16-01-30
10-01-16-03-12
10-01-16-03-50
10-01-16-10-18
10-01-16-13-57
10-01-16-20-28
10-06-18-27-51
10-07-16-37-14
10-07-16-51-08
10-08-10-26-23
10-08-10-34-37
10-08-10-39-20
10-08-10-45-54
no_lidar
resolution_failure
684


  0%|          | 0/684 [00:00<?, ?it/s]

/bean/depth/09-05-17-07-36/0.hdf5
/bean/depth/09-05-17-07-36/1.hdf5
/bean/depth/09-05-17-07-36/2.hdf5
/bean/depth/09-05-17-07-36/3.hdf5
/bean/depth/09-05-17-07-36/4.hdf5
/bean/depth/09-05-17-07-36/5.hdf5
/bean/depth/09-05-17-07-36/6.hdf5
/bean/depth/09-05-17-07-36/7.hdf5
/bean/depth/09-05-17-07-36/8.hdf5
/bean/depth/09-05-17-07-36/9.hdf5
/bean/depth/09-05-17-07-36/10.hdf5
/bean/depth/09-05-17-07-36/11.hdf5
/bean/depth/09-05-17-10-57/0.hdf5
/bean/depth/09-05-17-27-03/0.hdf5
/bean/depth/09-05-17-27-17/0.hdf5
/bean/depth/09-05-17-27-17/1.hdf5
/bean/depth/09-05-17-27-17/2.hdf5
/bean/depth/09-05-17-27-17/3.hdf5
/bean/depth/09-05-17-27-17/4.hdf5
/bean/depth/09-05-17-27-17/5.hdf5
/bean/depth/09-05-17-29-14/0.hdf5
/bean/depth/09-05-17-32-37/0.hdf5
/bean/depth/09-05-17-34-15/0.hdf5
/bean/depth/09-05-17-34-15/1.hdf5
/bean/depth/09-05-17-34-36/0.hdf5
/bean/depth/09-05-17-34-36/1.hdf5
/bean/depth/09-05-17-34-36/2.hdf5
/bean/depth/09-05-17-34-36/3.hdf5
/bean/depth/09-05-17-34-36/4.hdf5
/bean/depth/

  0%|          | 0/100 [00:00<?, ?it/s]

Exposure Error : 43 LidarAlignError : 92
/bean/depth/10-06-18-27-51/1.hdf5


...

Exposure Error : 50 LidarAlignError : 32
/bean/depth/10-08-10-45-54/54.hdf5


  0%|          | 0/100 [00:00<?, ?it/s]

Exposure Error : 60 LidarAlignError : 84
/bean/depth/10-08-10-45-54/55.hdf5


  0%|          | 0/94 [00:00<?, ?it/s]

Exposure Error : 44 LidarAlignError : 71


In [5]:
os.chdir("..")

In [ ]:
import os
import sys
sys.path.append("CostDCNet")


from task import Sub_trainer
from options import Options
os.chdir("CostDCNet")
opts = Options().parse()
trainer = Sub_trainer(opts)
trainer.evaluate(is_offline=True)
os.chdir("..")






In [ ]:
from myutils.depth import get_depth_anything_model
from typing import Union
depth_anything = get_depth_anything_model()
import numpy as np
from myutils.image_process import cv2toTensor
import torch

def inference_depth(image: Union[np.ndarray, torch.Tensor]):
    if isinstance(image, np.ndarray):
        image = cv2toTensor(image).cuda()
    depth = depth_anything(image)
    return depth

In [ ]:
import h5py
from tqdm.notebook import tqdm
import os
import cv2
from myutils.hy5py import calibration_property, read_calibration
from myutils.points import (
    combine_disparity_by_lidar,
    project_points_on_camera,
    transform_point_inverse,
)
import matplotlib.pyplot as plt

transform_mtx = np.load("jai_transform.npy")


def process_frame_disparity_refine(frame: h5py.Group, h5file: str, calibration):
    scene_folder = os.path.dirname(h5file)
    image_left_path = os.path.join(scene_folder, frame["image"].attrs["rgb_left_path"])
    image_left_nir_path = os.path.join(
        scene_folder, frame["image"].attrs["nir_left_path"]
    )
    image_left = (
        cv2toTensor(cv2.imread(image_left_path, cv2.IMREAD_COLOR)).cuda() / 255.0
    )
    image_left_nir = (
        cv2toTensor(cv2.imread(image_left_nir_path, cv2.IMREAD_GRAYSCALE))
        .cuda()
        .repeat(1, 3, 1, 1)
        / 255.0
    )
    image_left = torch.cat([image_left, image_left_nir], dim=0)
    depth = inference_depth(image_left)
    depth = depth.cpu().numpy().squeeze()
    depth_rgb, depth_nir = depth
    if "depth_mono/rgb" in frame:
        del frame["depth_mono/rgb"]
    if "depth_mono/nir" in frame:
        del frame["depth_mono/nir"]
    frame.create_dataset("depth_mono/rgb", data=depth_rgb)
    frame.create_dataset("depth_mono/nir", data=depth_nir)


def process_h5file_disparity_refine(h5file: str):
    with h5py.File(h5file, "a") as f:
        calibration = calibration_property(read_calibration(f))
        for frame_id in tqdm(f["frame"]):
            frame = f["frame"][frame_id]
            process_frame_disparity_refine(frame, h5file, calibration)
        f.close()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import h5py
import torch
import os
from myutils.depth import get_depth_anything_model
from myutils.image_process import cv2toTensor
from myutils.points import pad_lidar_points, refine_disparity_with_monodepth, transform_point_inverse, project_points_on_camera, combine_disparity_by_lidar
from tqdm.notebook import tqdm
from myutils.hy5py import calibration_property, read_calibration
depth_anything = get_depth_anything_model()
transform_mtx = np.load("jai_transform.npy")

print(transform_mtx)
def process_h5file_disparity_merge(h5file: str, at=0):
    with h5py.File(h5file, "a") as f:
        calibration = calibration_property(read_calibration(f))
        fx, bs, cx, cy = calibration
        idx = 0
        for frame_id in tqdm(f["frame"]):
            if idx < at:
                idx += 1
                continue
            frame = f["frame"][frame_id]
            disparity_rgb = frame["disparity/rgb"][:]
            disparity_nir = frame["disparity/nir"][:]
            image_left = cv2.imread(os.path.join(os.path.dirname(h5file), frame_id, "rgb", "left.png"))
            monodepth_rgb = frame["depth_mono/rgb"][:]
            monodepth_nir = frame["depth_mono/nir"][:]
            #monodepth_rgb = depth_anything(cv2toTensor(image_left).cuda()).cpu()[0].numpy()
            image_left_nir = cv2.imread(os.path.join(os.path.dirname(h5file), frame_id, "nir", "left.png"))
            #monodepth_nir = depth_anything(cv2toTensor(image_left_nir).cuda()).cpu()[0].numpy()
            disparity_rgb = refine_disparity_with_monodepth(
                disparity_rgb, monodepth_rgb
            )
            disparity_nir = refine_disparity_with_monodepth(
                disparity_nir, monodepth_nir
            )
            lidar_points = frame["lidar/points"][:] * 1000
            lidar_points = transform_point_inverse(lidar_points, transform_mtx)
            lidar_points = project_points_on_camera(lidar_points, fx, cx, cy, 720,540)
            lidar_points[:,2 ] = bs * fx / lidar_points[:,2] - 1
            lidar_points = pad_lidar_points(lidar_points, 5000)
            
            disparity = combine_disparity_by_lidar(lidar_points, disparity_rgb, disparity_nir, 48, 48)
            
            break
        f.close()
    plt.figure(figsize=(15,5))
    plt.subplot(131)
    plt.imshow(disparity_rgb, cmap="magma", vmin = 0, vmax = 16)
    plt.subplot(132)
    plt.imshow(disparity_nir, cmap="magma", vmin = 0, vmax = 16)
    plt.subplot(133)
    plt.imshow(disparity, cmap="magma", vmin = 0, vmax = 16)
    plt.show()
    plt.figure(figsize=(15,5))
    plt.subplot(131)
    plt.imshow(monodepth_rgb, cmap="magma", vmin = 0, vmax = 16)
    plt.subplot(132)
    plt.imshow(monodepth_nir, cmap="magma", vmin = 0, vmax = 16)
    
    

In [6]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

# 가우시안 잡음의 표준 편차
sigma = 1.0

# 윈도우 크기 (현재 예시에서는 각 픽셀을 독립적으로 처리)
window_size = 5

def initialize(A, B):
    F = (A + B) / 2
    alpha = np.zeros(A.shape + (2,), dtype=int)  # 마지막 차원: A, B
    beta = np.zeros(A.shape + (2,))
    return F, alpha, beta

def e_step(A, B, F, beta, sigma):
    # \alpha의 가능한 값: 1, 0, -1
    alphas = np.array([1, 0, -1])
    
    # Initialize responsibility arrays
    gamma_A = np.zeros(A.shape + (len(alphas),))
    gamma_B = np.zeros(B.shape + (len(alphas),))
    
    # Compute likelihoods for each possible alpha
    for idx, alpha_val in enumerate(alphas):
        # For image A
        L_A = (1 / np.sqrt(2 * np.pi * sigma**2)) * \
              np.exp(-((A - alpha_val * F - beta[..., 0])**2) / (2 * sigma**2))
        gamma_A[..., idx] = L_A
        
        # For image B
        L_B = (1 / np.sqrt(2 * np.pi * sigma**2)) * \
              np.exp(-((B - alpha_val * F - beta[..., 1])**2) / (2 * sigma**2))
        gamma_B[..., idx] = L_B
    
    # Sum over all possible alphas for normalization
    sum_gamma = gamma_A.sum(axis=-1) + gamma_B.sum(axis=-1) + 1e-8  # Avoid division by zero
    
    # Compute posterior probabilities
    gamma_A /= sum_gamma[..., np.newaxis]
    gamma_B /= sum_gamma[..., np.newaxis]
    
    return gamma_A, gamma_B

def m_step(A, B, gamma_A, gamma_B, alphas):
    # alphas: (3,) -> reshape to (1, 1, 3) for broadcasting with (540,720,3)
    alphas_reshaped = alphas[np.newaxis, np.newaxis, :]  # (1, 1, 3)
    
    # 확장된 A와 B
    A_expanded = A[..., np.newaxis]  # (540, 720, 1)
    B_expanded = B[..., np.newaxis]  # (540, 720, 1)
    
    # Update F
    numerator = (gamma_A * alphas_reshaped * A_expanded +
                 gamma_B * alphas_reshaped * B_expanded)
    denominator = (gamma_A + gamma_B) * alphas_reshaped + 1e-8  # Avoid division by zero
    
    # Compute F as weighted average
    F_new = numerator.sum(axis=-1) / denominator.sum(axis=-1)
    
    # Update beta
    beta_A = (A_expanded - F_new[..., np.newaxis] * alphas_reshaped) * gamma_A
    beta_B = (B_expanded - F_new[..., np.newaxis] * alphas_reshaped) * gamma_B
    
    # Sum over alphas
    beta_A = beta_A.sum(axis=-1) / (gamma_A.sum(axis=-1) + 1e-8)
    beta_B = beta_B.sum(axis=-1) / (gamma_B.sum(axis=-1) + 1e-8)
    
    # Update alpha by selecting the alpha with the highest posterior probability
    alpha_A = alphas[np.argmax(gamma_A, axis=-1)]
    alpha_B = alphas[np.argmax(gamma_B, axis=-1)]
    
    return F_new, alpha_A, alpha_B, beta_A, beta_B

def em_algorithm(A, B, max_iters=100, tol=1e-3):
    F, alpha, beta = initialize(A, B)
    alphas = np.array([1, 0, -1])
    
    for iteration in range(max_iters):
        F_old = F.copy()
        
        # E 단계
        gamma_A, gamma_B = e_step(A, B, F, beta, sigma)
        
        # M 단계
        F, alpha_A, alpha_B, beta_A, beta_B = m_step(A, B, gamma_A, gamma_B, alphas)
        beta = np.stack([beta_A, beta_B], axis=-1)
        
        # 알파 업데이트
        alpha[..., 0] = alpha_A
        alpha[..., 1] = alpha_B
        
        # 수렴 검사
        delta = np.linalg.norm(F - F_old)
        print(f"Iteration {iteration+1}, delta={delta:.6f}")
        if delta < tol:
            print("수렴 완료.")
            break
    return F, alpha, beta


In [36]:
import cv2
import numpy as np
from skimage import exposure
def calculate_cdf(hist):
    cdf = hist.cumsum()
    cdf_normalized = cdf / cdf[-1]
    return cdf_normalized

def estimate_exposure_ratio(gray1, gray2):
    # 평균 밝기 계산
    mean1 = np.mean(gray1)
    mean2 = np.mean(gray2)
    

    exposure_ratio = mean2 / mean1
    
    
    # 비율을 로그 값으로 변환 (HDR 알고리즘에서 사용)
    
    log_exposure_ratio = np.log2(exposure_ratio)
    
    return log_exposure_ratio
def compute_hdr(img1, img2):
    # if img1.mean() > img2.mean():
    #     img1, img2 = img2, img1
    print("exposure time")
    exposure_ratio = estimate_exposure_ratio(img1, img2)
    print(exposure_ratio)
    exposure_times = np.array([1.0,2.0**exposure_ratio], dtype=np.float32)
    img1 = np.repeat(img1[..., np.newaxis] * 255, 3, axis=-1).astype(np.uint8)
    img2 = np.repeat(img2[..., np.newaxis] * 255, 3, axis=-1).astype(np.uint8)
    print(img1.shape, img2.shape)
    images = [img1, img2]
    print(exposure_times)
    merge_debevec =  cv2.createMergeRobertson()
    #exposure_times=[1.0, 2.0]
    hdr = merge_debevec.process(images, times=exposure_times)
    print("hdr")
    return hdr

def local_histogram_matching(source_gray, template_gray, clip_limit=2.0, tile_grid_size=(24,24), exposure_limit=0.1):
    source_gray = (source_gray * 255).astype(np.uint8)
    template_gray = (template_gray * 255).astype(np.uint8)
    # CLAHE 적용하여 대비 향상
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    source_clahe = clahe.apply(source_gray)
    template_clahe = clahe.apply(template_gray)
    
    # 히스토그램 매칭 수행
    matched = exposure.match_histograms(source_clahe, template_clahe)
    matched = cv2.fastNlMeansDenoising(matched.astype(np.uint8), None, h=10, templateWindowSize=15, searchWindowSize=21)
    matched[template_gray < exposure_limit * 255] = source_gray[template_gray < exposure_limit * 255]
    return matched / 255

def false_fusion_algorithm_2(rgb: np.ndarray, nir: np.ndarray):
    yuv_mat = np.array([[0.299, 0.587, 0.114], [-0.147, -0.289, 0.436], [0.615, -0.515, -0.100]])
    yuv = np.dot(rgb, yuv_mat.T)
    y, u, v = yuv[:,:,0], yuv[:,:,1], yuv[:,:,2]
    yh = y
    yh = compute_hdr(y,nir).mean(axis=-1)
    yh2 = nir
    # yh2 = (yh + y) / 2
    # yh = (yh + nir) / 2
    
    #yh = local_histogram_matching(y, nir)
    u = u * yh / y

    v = v * yh / y
    y_old = y
    
    print(yh.shape, nir.shape)
    #yn = yh
    mul = 25
    y_em, alpha, beta = em_algorithm(yh * mul , yh2 * mul, max_iters = 25)
    y_em /= mul
    
    #y_em = yh
    print(yh.max())
    
    print("eval y", yh.mean(), y.mean(), y_em.mean(), nir.mean())
    un = u  - nir
    vn = nir - v
    y = yh
    
    yn = np.sqrt(np.var(y)) / np.sqrt(np.var(y_em)) * (y_em - np.mean(y_em)) + np.mean(y)
    un = np.sqrt(np.var(u)) / np.sqrt(np.var(un)) * (un - np.mean(un)) + np.mean(u)
    
    d = np.abs(nir - np.mean(nir))
    mu = d / np.mean(d)
    vn = mu * np.sqrt(np.var(v)) / np.sqrt(np.var(vn)) * (vn - np.mean(vn)) + np.mean(v)
    
    yuv = np.stack([yn,un,vn], axis=2)
    rgb_mat = np.array([[1, 0, 1.13983], [1, -0.39465, -0.58060], [1, 2.03211, 0]])
    rgb = np.dot(yuv, rgb_mat.T)
    return rgb, [np.dot(np.stack([y_em,un, vn], axis=2), rgb_mat.T), yh]

In [ ]:
import os
from myutils.image_process import cv2toTensor
from myutils.color_fusion import false_fusion_algorithm
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt
frame_folder = "/bean/depth/09-28-21-15-50/21_31_01_725"
#frame_folder = "/bean/depth/09-08-17-27-33/17_35_36_287"

#frame_folder = "/bean/depth/09-28-17-34-59/17_36_28_144"
#frame_folder = "/bean/depth/09-28-17-34-59/17_35_46_519"
#frame_folder = "/bean/depth/09-28-21-15-50/21_34_03_125"
#frame_folder = "/bean/depth/09-08-17-27-33/17_33_16_354"
rgb_left = cv2.imread(os.path.join(frame_folder, "rgb", "left.png"))
rgb_left = cv2.cvtColor(rgb_left, cv2.COLOR_BGR2RGB).astype(np.float32) / 255
nir_left = cv2.imread(os.path.join(frame_folder, "nir", "left.png"), cv2.IMREAD_GRAYSCALE).astype(np.float32) / 255
fusion, [y_h, y_em] = false_fusion_algorithm_2(rgb_left, nir_left)
plt.figure(figsize=(20,8))
plt.subplot(131)
sc = plt.imshow(y_h, vmin=0, vmax=1)
plt.subplot(132)
sc = plt.imshow(y_em, vmin=0, vmax=1)
plt.show()
plt.figure(figsize=(20,8))
plt.subplot(131)
plt.imshow(rgb_left)
plt.subplot(132)
plt.imshow(nir_left, cmap="gray")
plt.subplot(133)
plt.imshow(fusion)
plt.show()
rgb_right = cv2.imread(os.path.join(frame_folder, "rgb", "right.png"))
nir_right = cv2.imread(os.path.join(frame_folder, "nir", "right.png"), cv2.IMREAD_GRAYSCALE) / 255
rgb_right = cv2.cvtColor(rgb_right, cv2.COLOR_BGR2RGB).astype(np.float32) / 255

fusion_right, _ = false_fusion_algorithm_2(rgb_right, nir_right)

plt.figure(figsize=(20,8))
plt.subplot(131)
plt.imshow(rgb_right)
plt.subplot(132)
plt.imshow(nir_right, cmap="gray")
plt.subplot(133)
plt.imshow(fusion_right)
plt.show()

with torch.no_grad():
    _, flow = model(cv2toTensor(rgb_left * 255).cuda(), cv2toTensor(rgb_right * 255).cuda(), test_mode = True)
    flow = -flow.cpu()[0].numpy().squeeze()
    _, flow_nir = model(cv2toTensor(nir_left * 255).cuda().repeat(1,3,1,1), cv2toTensor(nir_right * 255).cuda().repeat(1,3,1,1), test_mode = True)
    flow_nir = -flow_nir.cpu()[0].numpy().squeeze()
    _, flow_fusion = model(cv2toTensor(fusion * 255).cuda(), cv2toTensor(fusion_right * 255).cuda(), test_mode = True)
    flow_fusion = -flow_fusion.cpu()[0].numpy().squeeze()
print(nir_right.min(), nir_right.max())
plt.figure(figsize=(20,8))
plt.subplot(131)
plt.imshow(flow, cmap="magma", vmin=0, vmax=32)
plt.subplot(132)

plt.imshow(flow_nir, cmap="magma", vmin=0, vmax=32)
plt.subplot(133)
plt.imshow(flow_fusion, cmap="magma", vmin=0, vmax=32)
plt.show()


In [ ]:
process_h5file_disparity_merge("/bean/depth/09-05-17-07-26/0.hdf5",0)

In [ ]:
import cv2
depth = inference_depth(cv2.imread("/bean/depth/09-28-17-06-04/17_10_22_823/nir/left.png"))
import matplotlib.pyplot as plt
plt.imshow(depth.cpu().numpy()[0], cmap='magma')

In [ ]:
import h5py
from myutils.hy5py import read_calibration

with h5py.File("/bean/depth/09-09-19-46-45/0.hdf5", "r") as f:
    calibration = f["calibration"].attrs
    print(calibration["mtx_left"])
    print(calibration["mtx_right"])

2310 - 2160 150

In [ ]:
FOLDER = "/bean/depth/09-09-19-46-45"
h5files = os.listdir(FOLDER)
h5files = [x for x in h5files if x.endswith(".hdf5")]
FRAME_COUNT = 25
__rsme_rgb = []
__rsme_nir = []
__mae_rgb = []
__mae_nir = []
__rsme_rgb_per_frame = []
__rsme_nir_per_frame = []
__mae_rgb_per_frame = []
__mae_nir_per_frame = []
for h5file in tqdm(h5files):
    h5file = os.path.join(FOLDER, h5file)
    
    with h5py.File(h5file, "r") as f:
        # print(f.attrs["rsme_rgb"], f.attrs["rsme_nir"])
        # print(f.attrs["mae_rgb"], f.attrs["mae_nir"])
        # __rsme_rgb.append(f.attrs["rsme_rgb"])
        # __rsme_nir.append(f.attrs["rsme_nir"])
        # __mae_rgb.append(f.attrs["mae_rgb"])
        # __mae_nir.append(f.attrs["mae_nir"])
        for frame in f["frame"]:
            frame = f.require_group(f"frame/{frame}")
            __rsme_rgb_per_frame.append(frame["disparity/rgb"].attrs["rsme"])
            __rsme_nir_per_frame.append(frame["disparity/nir"].attrs["rsme"])
            __mae_rgb_per_frame.append(frame["disparity/rgb"].attrs["mae"])
            __mae_nir_per_frame.append(frame["disparity/nir"].attrs["mae"])
            if len(__rsme_rgb_per_frame) > FRAME_COUNT:
                __rsme_rgb.append(np.mean(__rsme_rgb_per_frame))
                __rsme_nir.append(np.mean(__rsme_nir_per_frame))
                __mae_rgb.append(np.mean(__mae_rgb_per_frame))
                __mae_nir.append(np.mean(__mae_nir_per_frame))
                __rsme_rgb_per_frame = []
                __rsme_nir_per_frame = []
                __mae_rgb_per_frame = []
                __mae_nir_per_frame = []
        f.close()




plt.figure(figsize=(10, 10))

plt.subplot(1, 2, 1)
plt.plot(np.arange(len(__rsme_rgb)) * FRAME_COUNT, __rsme_rgb, label="RGB")
plt.plot(np.arange(len(__rsme_nir)) * FRAME_COUNT, __rsme_nir, label="NIR")
plt.xlabel("Frame")
plt.ylabel("RMSE")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(np.arange(len(__mae_rgb)) * FRAME_COUNT, __mae_rgb, label="RGB")
plt.plot(np.arange(len(__mae_nir)) * FRAME_COUNT, __mae_nir, label="NIR")
plt.xlabel("Frame")
plt.ylabel("MAE")
plt.legend()

# plt.subplot(2, 2, 3)
# plt.plot(__rsme_rgb_per_frame, label="RSME RGB per frame")
# plt.plot(__rsme_nir_per_frame, label="RSME NIR per frame")
# plt.xlabel("Frame")
# plt.ylabel("RSME")
# plt.legend()

# plt.subplot(2, 2, 4)
# plt.plot(__mae_rgb_per_frame, label="MAE RGB per frame")
# plt.plot(__mae_nir_per_frame, label="MAE NIR per frame")
# plt.xlabel("Frame")
# plt.ylabel("MAE")
# plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))

plt.subplot(2, 2, 1)
plt.plot(__rsme_rgb, label="RSME RGB")
plt.plot(__rsme_nir, label="RSME NIR")
plt.xlabel("Scene")
plt.ylabel("RSME")
plt.ylim(0, 10)  # Set the y-axis range
plt.legend()

plt.subplot(2, 2, 2)
plt.plot(__mae_rgb, label="MAE RGB")
plt.plot(__mae_nir, label="MAE NIR")
plt.xlabel("Scene")
plt.ylabel("MAE")
plt.ylim(0, 5)  # Set the y-axis range
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(__rsme_rgb_per_frame, label="RSME RGB per frame")
plt.plot(__rsme_nir_per_frame, label="RSME NIR per frame")
plt.xlabel("Frame")
plt.ylabel("RSME")
plt.ylim(0, 10)  # Set the y-axis range
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(__mae_rgb_per_frame, label="MAE RGB per frame")
plt.plot(__mae_nir_per_frame, label="MAE NIR per frame")
plt.xlabel("Frame")
plt.ylabel("MAE")
plt.ylim(0, 5)  # Set the y-axis range
plt.legend()

plt.tight_layout()
plt.show()

In [40]:
from tqdm.notebook import tqdm
import h5py
import cv2
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
def evaluate_exposure(image):
    if image.mean() < 12:
        return -1
    if image.mean() > 196:
        return 1
    return 0

def evaluate_exposure_stereo(image_left, image_right):
    left_intensity = image_left.mean()
    right_intensity = image_right.mean()
    if left_intensity / right_intensity > 1.15:
        return 1
    if right_intensity / left_intensity > 1.15:
        return -1
    return 0



def fft_blur_detection(image):
    """
    Detect blur in an image using FFT (Fast Fourier Transform).
    Steps:
    1. Convert image to grayscale.
    2. Apply Fourier Transform.
    3. Analyze high frequency components to determine blur level.
    
    Parameters:
    - image: Input BGR image
    
    Returns:
    - blur_metric: A scalar representing the amount of blur in the image.
                   Lower values indicate more blur.
    """
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape) == 3 else image
    
    # Get the image dimensions
    (h, w) = gray.shape
    
    # Apply a Fast Fourier Transform (FFT)
    f_transform = np.fft.fft2(gray)
    f_shifted = np.fft.fftshift(f_transform)
    
    # Compute the magnitude spectrum
    magnitude_spectrum = 20 * np.log(np.abs(f_shifted))
    
    # Crop the magnitude spectrum to remove the low frequencies
    # Center coordinates
    center_row, center_col = h // 2, w // 2
    # Radius of the low frequency region to remove
    radius = 60
    
    # Create a mask to zero out the low frequencies
    mask = np.ones((h, w), np.uint8)
    cv2.circle(mask, (center_col, center_row), radius, 0, -1)
    
    # Apply the mask to the magnitude spectrum
    f_shifted_masked = f_shifted * mask
    
    # Compute the inverse FFT
    f_ishift = np.fft.ifftshift(f_shifted_masked)
    reconstructed_image = np.fft.ifft2(f_ishift)
    reconstructed_image = np.abs(reconstructed_image)
    
    # Compute the variance of the reconstructed image (sharpness measure)
    blur_metric = np.var(reconstructed_image)
    
    return blur_metric


def lidar_box_error(lidar_points, disparity_map, box_size=5):
    """
    벡터화된 방식으로 LiDAR 포인트와 Disparity Map을 이용하여 Box Error를 계산합니다.

    Parameters:
    - lidar_points: (N, 3) 배열로 각 행이 (u, v, d)를 나타냅니다.
    - disparity_map: 2D 배열로 Disparity Map을 나타냅니다.
    - box_size: 박스의 크기 (기본값은 5).

    Returns:
    - mean_box_error: 모든 포인트에 대한 평균 Box Error.
    - errors: 각 포인트에 대한 [u, v, box_error] 배열.
    """
    # LiDAR 포인트 분리
    u, v, d = lidar_points.T
    h, w = disparity_map.shape
    half_box = box_size // 2

    # Disparity Map을 패딩하여 경계 문제를 해결
    padded_disparity = np.pad(disparity_map, 
                              pad_width=((half_box, half_box), (half_box, half_box)), 
                              mode='edge')

    # 슬라이딩 윈도우을 이용해 모든 5x5 박스를 생성
    windows = sliding_window_view(padded_disparity, (box_size, box_size))
    # windows의 shape: (h, w, box_size, box_size)

    # u와 v가 이미지 경계를 벗어나지 않도록 클리핑
    u_clipped = np.clip(u, 0, w - 1).astype(int)
    v_clipped = np.clip(v, 0, h - 1).astype(int)

    # 각 포인트에 해당하는 박스 추출
    # indexing을 통해 (N, box_size, box_size) 형태의 배열을 얻음
    boxes = windows[v_clipped, u_clipped, :, :]

    # 실제 d 값과 박스 내 값들의 절대 차이 계산
    diffs = np.abs(boxes - d[:, np.newaxis, np.newaxis])

    # 박스 에러 (절대 차이의 평균) 계산
    box_errors = np.mean(diffs, axis=(1, 2))

    # NaN 값이 있는 경우 제거
    valid = ~np.isnan(box_errors)
    box_errors = box_errors[valid]
    u_valid = u_clipped[valid]
    v_valid = v_clipped[valid]

    # 전체 평균 박스 에러 계산
    mean_box_error = np.mean(box_errors)

    # 각 포인트의 [u, v, box_error] 배열 생성
    errors = np.stack([u_valid, v_valid, box_errors], axis=1)

    return mean_box_error, errors
        
    
    

def evaluate_frame_exposure(frame: h5py.Group, frame_path: str):
    images = read_image_pair(frame_path)
    exposure_status = [evaluate_exposure(image) for image in images]
    stereo_status = [evaluate_exposure_stereo(images[0], images[1]), evaluate_exposure_stereo(images[2], images[3])]
    blur_status = [fft_blur_detection(image) for image in images]
    if (exposure_status[0] != 0 and exposure_status[2] != 0) or stereo_status[0] != 0 or stereo_status[1] != 0:
        frame.attrs["exposure_error"] = True
    else:
        frame.attrs["exposure_error"] = False

    if blur_status[2] < 120 or blur_status[3] < 120:
        frame.attrs["blur_error"] = True
    else:
        frame.attrs["blur_error"] = False

    

def update_frame_exposure_h5(h5file: str):
    with h5py.File(h5file, "a") as f:
        for frame_id in tqdm(f["frame"]):
            frame = f["frame"][frame_id]
            evaluate_frame_exposure(frame, os.path.join(os.path.dirname(h5file), frame_id))
        f.close()    


    

In [33]:
from myutils.points import lidar_points_to_disparity_with_cal, refine_disparity_points
from myutils.hy5py import  read_calibration, get_frame_by_path
from myutils.image_process import read_image_pair
import numpy as np
import os
import matplotlib.pyplot as plt
import torch



def evaluate_frame(frame_path: str):
    t_mtx = np.load("jai_transform.npy")
    cal = read_calibration(os.path.dirname(frame_path) + "/0.hdf5")
    images = read_image_pair(frame_path)
    with get_frame_by_path(frame_path) as frame:
        if "rgb_exposure_left" in frame["image"].attrs:
            for s in ["rgb","nir"]:
                for d in ["left","right"]:
                    if f"{s}_exposure_{d}" in frame["image"].attrs:
                        print(f"{s} {d} exposure: ", frame["image"].attrs[f"{s}_exposure_{d}"])
        lidar_disparity = lidar_points_to_disparity_with_cal(frame["lidar/points"][:], t_mtx , cal)
        lidar_disparity = refine_disparity_points(torch.from_numpy(lidar_disparity),).numpy()
        if "disparity/rgb" in frame:
            disparity_rgb = frame["disparity/rgb"][:]
            disparity_nir = frame["disparity/nir"][:]

        
    lidar_error, lidar_e = lidar_box_error(lidar_disparity , disparity_rgb)
    lidar_nir_error, _ = lidar_box_error(lidar_disparity , disparity_nir)
    exposure_times = [i.mean() for i in images]
    print(exposure_times)
    exposure_status = [evaluate_exposure(image) for image in images]
    stereo_status = [evaluate_exposure_stereo(images[0], images[1]), evaluate_exposure_stereo(images[2], images[3])]
    blur_status = [fft_blur_detection(image) for image in images]
    print("Lidar Box Error ",lidar_error, lidar_nir_error)
    print(blur_status)
    for i, exposure_status in enumerate(exposure_status):
        print(f"{'rgb' if i//2 == 0 else 'nir'} {'left' if i%2 == 0 else 'right'}: ", end="")
        if exposure_status == -1:
            print("Underexposed")
        elif exposure_status == 1:
            print("Overexposed")
        else:
            print("Normal")
    for i, stereo_status in enumerate(stereo_status):
        print(f"{'rgb' if i == 0 else 'nir'} stereo: ", end="")
        if stereo_status == -1:
            print("Right Overexposed")
        elif stereo_status == 1:
            print("Left Overexposed")
        else:
            print("Normal")
            
    plt.figure(figsize=(15,5))
    plt.subplot(131)
    plt.imshow(disparity_rgb, cmap="rainbow", vmin=0, vmax=32)
    plt.subplot(132)
    plt.imshow(disparity_nir, cmap="rainbow", vmin=0, vmax=32)
    plt.subplot(133)
    plt.imshow(images[0])
    plt.scatter(lidar_e[:,0], lidar_e[:,1], c=lidar_e[:,2], cmap="rainbow", s=0.5, vmin=0, vmax=8)
    plt.show()
    
    
    

In [41]:
from myutils.widget import FrameExplorer


FrameExplorer(evaluate_frame)